In [25]:
from pandas import DataFrame as df
from pandas import read_csv
from pandas import concat
from pandas import to_datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
TestDataset = read_csv('./test.csv')
TrainDataset = read_csv('./train.csv')

In [27]:
TestDataframe= df(TestDataset)
TrainDataset= df(TrainDataset)

#Checking for missing values
print(TrainDataset.isnull().sum())
print(TestDataframe.isnull().sum())




PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             37
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          134
Embarked         0
dtype: int64


In [28]:
TrainDataset=TrainDataset.dropna()
print(TrainDataset.head())

    PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
1             2         1       1  ...  71.2833   C85         C
3             4         1       1  ...  53.1000  C123         S
6             7         0       1  ...  51.8625   E46         S
10           11         1       3  ...  16.7000    G6         S
11           12         1       1  ...  26.5500  C103         S

[5 rows x 12 columns]


In [29]:
TrainDataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [30]:
TrainDataset.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [31]:
TrainDataset=TrainDataset.drop(['Name','Ticket','Cabin'],axis=1)

In [34]:
TestDataframe["FamilySize"] = TestDataframe["SibSp"] + TestDataframe["Parch"] + 1
TrainDataset["FamilySize"] = TrainDataset["SibSp"] + TrainDataset["Parch"] + 1

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [35]:
X = TrainDataset.drop(columns=['Survived', 'PassengerId'])


In [36]:
y= TrainDataset['Survived']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

numeric_features = ['Age', 'Fare', 'FamilySize']
cat_features=['Pclass','Sex','Embarked']
num_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                  ('scaler', StandardScaler())])
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numeric_features),
        ('cat', cat_transformer, cat_features)
    ])


model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy_score = model.score(X_test, y_test)
print(f'Model accuracy: {accuracy_score}')
report=classification_report(y_test, y_pred,target_names=["Did not survive", "Survived"])

print(report)

Model accuracy: 0.7837837837837838
                 precision    recall  f1-score   support

Did not survive       0.64      0.75      0.69        12
       Survived       0.87      0.80      0.83        25

       accuracy                           0.78        37
      macro avg       0.76      0.78      0.76        37
   weighted avg       0.80      0.78      0.79        37



In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler



In [40]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'Fare', 'FamilySize']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Pclass', 'Sex', 'Embarked'])
    ])



In [48]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

base_models_pipeline = [
  ('random_forest', Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=1000))
  ])),
  ('gradient_boosting', Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42, n_estimators=1000))
  ]))
]

stacking_pipeline = StackingClassifier(
  estimators=base_models_pipeline,
  final_estimator=LogisticRegression(),
  cv=5
)

X = TrainDataset.drop(columns=['Survived', 'PassengerId'])
y = TrainDataset['Survived']

# Using the entire dataset for both training and testing to overfit the model
stacking_pipeline.fit(X, y)
y_pred_stacking = stacking_pipeline.predict(X)
stacking_accuracy = accuracy_score(y, y_pred_stacking)

print(f'Stacking accuracy: {stacking_accuracy:.2f}')

Stacking accuracy: 1.00
